# QiskitRuntimeService - Code Laboratory

**Section 4: Running Circuits on IBM Quantum Hardware** | [See README for concepts](./README.md)

---

## 🔧 Quick API Reference

| Method | Signature | Returns | Use When |
|--------|-----------|---------|----------|
| `save_account()` | `QiskitRuntimeService.save_account(channel, token)` | `None` | One-time credential setup |
| `QiskitRuntimeService()` | `QiskitRuntimeService(channel=None, token=None)` | `QiskitRuntimeService` | Connect to IBM Quantum |
| `backends()` | `service.backends(**filters)` | `List[IBMBackend]` | Get list of available backends |
| `backend()` | `service.backend(name)` | `IBMBackend` | Get specific backend by name |
| `least_busy()` | `service.least_busy(**filters)` | `IBMBackend` | Auto-select shortest queue |

---

In [ ]:
"""
Qiskit Code Laboratory - QiskitRuntimeService
==============================================
Prerequisites: See README.md for conceptual background
"""

# Standard imports
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

# IBM Quantum Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler
from qiskit_ibm_runtime.fake_provider import FakeKyoto, FakeManilaV2

# =============================================================
# UTILITY FUNCTIONS FOR THIS NOTEBOOK
# =============================================================

def print_backend_info(backend, label="Backend"):
    """Display key backend properties."""
    print(f"\n{label}: {backend.name}")
    print(f"  Qubits: {backend.num_qubits}")
    print(f"  Operational: {getattr(backend, 'status', lambda: type('', (), {'operational': 'N/A'})()).operational if hasattr(backend, 'status') else 'N/A'}")
    if hasattr(backend, 'operation_names'):
        ops = list(backend.operation_names)[:5]
        print(f"  Operations (first 5): {ops}")

def is_simulator(backend):
    """Check if backend is a simulator."""
    return 'simulator' in backend.name.lower() if hasattr(backend, 'name') else False

print("✅ Environment ready - using FakeKyoto for demonstrations")

---

## `QiskitRuntimeService.save_account()`

### Signature
```python
QiskitRuntimeService.save_account(
    channel: str,           # 'ibm_quantum' (free) or 'ibm_cloud'
    token: str,             # Your API token
    instance: str = None,   # Optional: hub/group/project
    overwrite: bool = False,
    set_as_default: bool = True
) -> None
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `channel` | `str` | Yes | - | 'ibm_quantum' for free tier, 'ibm_cloud' for paid |
| `token` | `str` | Yes | - | Your IBM Quantum API token |
| `instance` | `str` | No | `None` | hub/group/project path |
| `overwrite` | `bool` | No | `False` | Replace existing credentials |
| `set_as_default` | `bool` | No | `True` | Set as default channel |

### Returns
`None` - Saves credentials to disk

### See Also
- [README: Service Initialization Patterns](./README.md#1-service-initialization-patterns)

In [ ]:
# ============================================================
# save_account() - BASIC USAGE
# ============================================================

# ONE-TIME SETUP (run once per machine)
# This saves credentials to disk so you don't need to provide token every time

# Pattern 1: Free tier (ibm_quantum)
# QiskitRuntimeService.save_account(
#     channel='ibm_quantum',
#     token='YOUR_API_TOKEN_HERE'
# )

# Pattern 2: Paid tier (ibm_cloud) with instance
# QiskitRuntimeService.save_account(
#     channel='ibm_cloud',
#     token='YOUR_API_KEY',
#     instance='hub/group/project'
# )

print("⚠️ save_account() is ONE-TIME setup")
print("After saving, use QiskitRuntimeService() without arguments")
print()
print("Example (don't run unless you have a real token):")
print('  QiskitRuntimeService.save_account(channel="ibm_quantum", token="YOUR_TOKEN")')
print()
print("✅ Credentials saved → Service loads them automatically")

In [ ]:
# ============================================================
# save_account() - PARAMETER VARIATIONS
# ============================================================

# Variation 1: Basic save (most common)
print("Variation 1: Basic free tier setup")
print('  QiskitRuntimeService.save_account(channel="ibm_quantum", token="...")')
print()

# Variation 2: Overwrite existing credentials
print("Variation 2: Replace existing credentials")
print('  QiskitRuntimeService.save_account(channel="ibm_quantum", token="...", overwrite=True)')
print()

# Variation 3: Multiple channels (save both)
print("Variation 3: Save multiple channels")
print('  # Save free tier')
print('  QiskitRuntimeService.save_account(channel="ibm_quantum", token="...", set_as_default=True)')
print('  # Save paid tier (doesn\'t overwrite free)')
print('  QiskitRuntimeService.save_account(channel="ibm_cloud", token="...", instance="...")')
print()

# Variation 4: With instance path
print("Variation 4: With hub/group/project instance")
print('  QiskitRuntimeService.save_account(')
print('      channel="ibm_cloud",')
print('      token="...",')
print('      instance="ibm-q-internal/deployment/default"')
print('  )')

print("\n✅ All variations shown")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: Channel Confusion
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: ibm_quantum vs ibm_cloud channels")
print("=" * 55)

print("\n❌ WRONG - Using wrong channel:")
print('  QiskitRuntimeService.save_account(channel="ibm_cloud", token="...")  # ← Free tier token!')
print("  # Error: Cloud channel requires different token format + instance")

print("\n✅ CORRECT - Matching channel to account type:")
print('  # Free account (from quantum.ibm.com):')
print('  QiskitRuntimeService.save_account(channel="ibm_quantum", token="...")')
print()
print('  # Paid account (from cloud.ibm.com):')
print('  QiskitRuntimeService.save_account(channel="ibm_cloud", token="...", instance="hub/group/project")')

print("\n🔍 KEY DIFFERENCES:")
print("  ibm_quantum: Free tier, no instance required, token from quantum.ibm.com")
print("  ibm_cloud:   Paid tier, instance REQUIRED, token from cloud.ibm.com")

print("\n💡 Remember: See README.md for mnemonic - 'Q for Quantum, C for Cloud, I for Instance!'")

---

## `QiskitRuntimeService()` Constructor

### Signature
```python
QiskitRuntimeService(
    channel: str = None,    # 'ibm_quantum' or 'ibm_cloud'
    token: str = None,      # Override saved token
    instance: str = None,   # Override saved instance
    **kwargs
) -> QiskitRuntimeService
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `channel` | `str` | No | saved default | Override channel selection |
| `token` | `str` | No | saved token | Inline token (not recommended) |
| `instance` | `str` | No | saved instance | Override instance path |

### Returns
`QiskitRuntimeService` - Connection to IBM Quantum services

### See Also
- [README: Three Initialization Patterns](./README.md#1-service-initialization-patterns)

In [ ]:
# ============================================================
# QiskitRuntimeService() - BASIC USAGE
# ============================================================

# Most common pattern: Load saved credentials
# service = QiskitRuntimeService()  # Uses credentials from save_account()

# For this notebook, we use FakeKyoto (no credentials needed)
fake_backend = FakeKyoto()
print("Using FakeKyoto for demonstrations (no IBM credentials required)")
print(f"Backend: {fake_backend.name}")
print(f"Qubits: {fake_backend.num_qubits}")

# The three initialization patterns:
print("\n" + "=" * 55)
print("THREE INITIALIZATION PATTERNS")
print("=" * 55)

print("\nPattern 1: Use saved credentials (MOST COMMON)")
print("  service = QiskitRuntimeService()")
print("  # Loads from save_account() - recommended!")

print("\nPattern 2: Explicit channel")
print('  service = QiskitRuntimeService(channel="ibm_quantum")')
print("  # Override default channel")

print("\nPattern 3: Inline token (not recommended)")
print('  service = QiskitRuntimeService(channel="ibm_quantum", token="...")')
print("  # Token in code - security risk!")

print("\n✅ Pattern 1 is the exam answer for 'how to initialize service'")

In [ ]:
# ============================================================
# QiskitRuntimeService() - VERIFICATION PATTERN
# ============================================================

def verify_service_pattern():
    """Test initialization patterns work correctly."""
    
    # Test 1: FakeBackend works as substitute
    backend = FakeKyoto()
    assert hasattr(backend, 'name'), "Backend should have name"
    assert hasattr(backend, 'num_qubits'), "Backend should have num_qubits"
    assert backend.num_qubits == 127, "FakeKyoto should have 127 qubits"
    
    # Test 2: Backend has required V2 API properties
    assert hasattr(backend, 'operation_names'), "V2 API: should have operation_names"
    assert hasattr(backend, 'coupling_map'), "V2 API: should have coupling_map"
    assert hasattr(backend, 'target'), "V2 API: should have target"
    
    # Test 3: Can get operations
    ops = backend.operation_names
    assert 'cx' in ops or 'ecr' in ops, "Should have 2-qubit gate"
    assert 'measure' in ops, "Should support measurement"
    
    print("✅ All service pattern tests passed!")

verify_service_pattern()

---

## `service.backends()`

### Signature
```python
service.backends(
    name: str = None,           # Filter by name pattern
    min_num_qubits: int = None, # Minimum qubits required
    simulator: bool = None,     # True/False/None for all
    operational: bool = None,   # Only working backends
    **kwargs
) -> List[IBMBackend]
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `name` | `str` | No | `None` | Backend name filter |
| `min_num_qubits` | `int` | No | `None` | Minimum qubit count |
| `simulator` | `bool` | No | `None` | Filter simulators |
| `operational` | `bool` | No | `None` | Only operational backends |

### Returns
`List[IBMBackend]` - List of matching backends

### See Also
- [README: Backend Selection Methods](./README.md#2-backend-selection)

In [ ]:
# ============================================================
# backends() - BASIC USAGE (using fake provider)
# ============================================================

# Import fake backends to demonstrate filtering
from qiskit_ibm_runtime.fake_provider import FakeManilaV2, FakeKyoto, FakeSherbrooke

# Create list of fake backends to simulate service.backends()
fake_backends = [FakeManilaV2(), FakeKyoto(), FakeSherbrooke()]

print("Available backends (simulated with FakeProvider):")
print("=" * 55)

for backend in fake_backends:
    print(f"  {backend.name}: {backend.num_qubits} qubits")

print("\n" + "=" * 55)
print("FILTERING PATTERNS")
print("=" * 55)

# Filter by qubit count
large_backends = [b for b in fake_backends if b.num_qubits >= 100]
print(f"\nBackends with >= 100 qubits: {[b.name for b in large_backends]}")

# Check for simulator (fake backends are simulators)
print("\nReal service filtering:")
print('  hardware_only = service.backends(simulator=False)')
print('  simulators = service.backends(simulator=True)')
print('  big_backends = service.backends(min_num_qubits=100)')

print("\n✅ backends() returns list - iterate or index to get individual backend")

In [ ]:
# ============================================================
# backends() - PARAMETER VARIATIONS
# ============================================================

print("PARAMETER VARIATIONS FOR service.backends()")
print("=" * 55)

# Variation 1: All backends
print("\n1. Get all backends:")
print('  all_backends = service.backends()')

# Variation 2: Filter by simulator
print("\n2. Filter simulators vs hardware:")
print('  simulators = service.backends(simulator=True)')
print('  hardware = service.backends(simulator=False)')

# Variation 3: Minimum qubits
print("\n3. Filter by qubit count:")
print('  large = service.backends(min_num_qubits=100)')
print('  small = service.backends(min_num_qubits=5, max_num_qubits=20)')

# Variation 4: Only operational
print("\n4. Only operational backends:")
print('  working = service.backends(operational=True)')

# Variation 5: Combined filters
print("\n5. Combined filters (AND logic):")
print('  # Large operational hardware')
print('  best = service.backends(')
print('      simulator=False,')
print('      operational=True,')
print('      min_num_qubits=100')
print('  )')

# Demonstrable with fake backends
print("\n" + "=" * 55)
print("LIVE DEMO with FakeProvider:")
demo_backends = [FakeManilaV2(), FakeKyoto()]
filtered = [b for b in demo_backends if b.num_qubits >= 100]
print(f"Backends with >= 100 qubits: {[b.name for b in filtered]}")

print("\n✅ All filter combinations shown")

---

## `service.backend()`

### Signature
```python
service.backend(
    name: str           # Exact backend name
) -> IBMBackend
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `name` | `str` | Yes | - | Exact backend name |

### Returns
`IBMBackend` - Single backend instance

### Raises
- `IBMNotAuthorizedError`: No access to backend
- `IBMBackendError`: Backend not found

### See Also
- [README: Backend Selection Methods](./README.md#2-backend-selection)

In [ ]:
# ============================================================
# backend() - BASIC USAGE
# ============================================================

# Get specific backend by name (using FakeKyoto as substitute)
backend = FakeKyoto()

print("service.backend() - Get specific backend by name")
print("=" * 55)

print(f"\nExample: backend = service.backend('ibm_kyoto')")
print(f"\nUsing FakeKyoto() for demo:")
print(f"  Name: {backend.name}")
print(f"  Qubits: {backend.num_qubits}")

# Key difference: backend() vs backends()
print("\n" + "=" * 55)
print("backend() vs backends() - KEY DIFFERENCE")
print("=" * 55)
print("\n  backends() → Returns LIST  → [backend1, backend2, ...]")
print("  backend()  → Returns SINGLE → backend_object")

print("\n💡 Exam tip: backend() requires exact name, backends() can filter")

print("\n✅ Use backend() when you know the exact backend name")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: backends() vs backend()
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Confusing backends() and backend()")
print("=" * 55)

# Demonstrate the difference
backend_list = [FakeManilaV2(), FakeKyoto()]  # Simulates service.backends()
single_backend = FakeKyoto()                   # Simulates service.backend('ibm_kyoto')

print("\n❌ WRONG - Treating list as single backend:")
print('  backends = service.backends()')
print('  backends.num_qubits  # ← AttributeError! List has no num_qubits')

print("\n✅ CORRECT - Index the list OR use backend():")
print('  # Option 1: Index the list')
print('  backends = service.backends()')
print('  backends[0].num_qubits  # ✓ Works')
print()
print('  # Option 2: Get single backend directly')
print('  backend = service.backend("ibm_kyoto")')
print('  backend.num_qubits  # ✓ Works')

# Live verification
print("\n🔍 LIVE VERIFICATION:")
print(f"  type(backend_list) = {type(backend_list)}")
print(f"  type(single_backend) = {type(single_backend)}")
print(f"  backend_list[0].num_qubits = {backend_list[0].num_qubits}")
print(f"  single_backend.num_qubits = {single_backend.num_qubits}")

print("\n💡 Remember: backends() = LIST (plural), backend() = SINGLE (singular)")

---

## `service.least_busy()`

### Signature
```python
service.least_busy(
    simulator: bool = None,     # True/False/None
    operational: bool = True,   # Only working backends
    min_num_qubits: int = None, # Minimum qubits
    **kwargs
) -> IBMBackend
```

### Parameters
| Parameter | Type | Required | Default | Description |
|-----------|------|----------|---------|-------------|
| `simulator` | `bool` | No | `None` | Filter simulators |
| `operational` | `bool` | No | `True` | Only operational backends |
| `min_num_qubits` | `int` | No | `None` | Minimum qubit count |

### Returns
`IBMBackend` - Backend with shortest job queue

### See Also
- [README: Backend Selection Methods](./README.md#2-backend-selection)

In [ ]:
# ============================================================
# least_busy() - BASIC USAGE
# ============================================================

print("service.least_busy() - Auto-select backend with shortest queue")
print("=" * 55)

# Standard patterns
print("\n1. Least busy real hardware (MOST COMMON):")
print('  backend = service.least_busy(simulator=False, operational=True)')

print("\n2. Least busy simulator:")
print('  backend = service.least_busy(simulator=True)')

print("\n3. Least busy with minimum qubits:")
print('  backend = service.least_busy(simulator=False, min_num_qubits=100)')

# Cannot demonstrate live (requires real service), but show expected behavior
print("\n" + "=" * 55)
print("EXPECTED BEHAVIOR:")
print("  → Queries all accessible backends")
print("  → Filters by simulator/operational/min_num_qubits")
print("  → Returns backend with shortest pending job queue")

print("\n💡 Exam tip: simulator=False excludes simulators for REAL hardware")
print("\n✅ Use least_busy() for automatic backend selection")

In [ ]:
# ============================================================
# least_busy() - PARAMETER VARIATIONS
# ============================================================

print("PARAMETER VARIATIONS FOR service.least_busy()")
print("=" * 55)

print("\n1. Default - any operational backend:")
print('  backend = service.least_busy()')
print('  # Includes both simulators and hardware')

print("\n2. Hardware only (excludes simulators):")
print('  backend = service.least_busy(simulator=False)')

print("\n3. With operational filter:")
print('  backend = service.least_busy(simulator=False, operational=True)')
print('  # Skips backends under maintenance')

print("\n4. Minimum qubit requirement:")
print('  backend = service.least_busy(min_num_qubits=27)')

print("\n5. Combined - production pattern:")
print('  backend = service.least_busy(')
print('      simulator=False,')
print('      operational=True,')
print('      min_num_qubits=27')
print('  )')

print("\n✅ Combined filters give best production backend")

In [ ]:
---

## Backend Properties (V2 API)

### Key Properties
| Property | Type | Description |
|----------|------|-------------|
| `name` | `str` | Backend identifier |
| `num_qubits` | `int` | Number of qubits |
| `operation_names` | `set[str]` | Supported gate names |
| `coupling_map` | `CouplingMap` | Qubit connectivity |
| `target` | `Target` | Detailed hardware specs |
| `dt` | `float` | System time resolution |

### V1 vs V2 API
```python
# V1 (DEPRECATED)
backend.configuration().basis_gates  # ❌ Old API

# V2 (CURRENT - Qiskit 1.0+)
backend.operation_names              # ✅ New API
```

### See Also
- [README: Backend Properties](./README.md#3-backend-properties)

In [ ]:
# ============================================================
# Backend Properties - LIVE EXPLORATION
# ============================================================

backend = FakeKyoto()

print("BACKEND PROPERTIES (V2 API)")
print("=" * 55)

# Essential properties
print(f"\n1. name: '{backend.name}'")
print(f"2. num_qubits: {backend.num_qubits}")

# Operations
ops = list(backend.operation_names)
print(f"3. operation_names: {ops[:8]}...")  # First 8

# Coupling map
cm = backend.coupling_map
print(f"4. coupling_map edges (first 5): {list(cm.get_edges())[:5]}...")

# Target
target = backend.target
print(f"5. target: {type(target).__name__} with {len(target.operations)} operations")

# dt (time resolution)
print(f"6. dt: {backend.dt} seconds")

print("\n" + "=" * 55)
print("⚠️ V1 vs V2 API COMPARISON")
print("=" * 55)

print("\n❌ V1 API (DEPRECATED):")
print('  backend.configuration().basis_gates')
print('  backend.configuration().n_qubits')

print("\n✅ V2 API (CURRENT):")
print('  backend.operation_names')
print('  backend.num_qubits')

print("\n💡 Exam always uses V2 API - ignore V1 patterns!")

In [ ]:
# ============================================================
# ⚠️ TRAP DEMONSTRATION: V1 vs V2 API
# This is what the exam might test!
# ============================================================

print("⚠️ TRAP: Using deprecated V1 API")
print("=" * 55)

backend = FakeKyoto()

print("\n❌ WRONG - V1 API (deprecated in Qiskit 1.0+):")
print('  # These will raise AttributeError on new backends:')
print('  backend.configuration().basis_gates')
print('  backend.configuration().n_qubits')
print('  backend.properties().qubits')

print("\n✅ CORRECT - V2 API:")
print('  backend.operation_names')
print('  backend.num_qubits')
print('  backend.target')

# Live verification
print("\n🔍 LIVE VERIFICATION:")
print(f"  backend.operation_names works: {bool(backend.operation_names)}")
print(f"  backend.num_qubits works: {backend.num_qubits}")
print(f"  backend.target works: {bool(backend.target)}")

# Show what V2 gives you
print("\n📋 V2 API PROPERTIES:")
print(f"  operation_names: {list(backend.operation_names)[:5]}...")
print(f"  num_qubits: {backend.num_qubits}")
print(f"  target.operations: {list(backend.target.operation_names)[:5]}...")

print("\n💡 Remember: V2 = simpler properties, V1 = configuration().method()")

In [ ]:
# ============================================================
# COMPARISON: Backend Selection Methods
# ============================================================

print("=" * 60)
print("COMPARING: backends() vs backend() vs least_busy()")
print("=" * 60)

# Create test data
fake_backends = [FakeManilaV2(), FakeKyoto()]

print("\n1. backends() - Returns LIST")
print("   Usage: service.backends(simulator=False)")
print(f"   Returns: {type(fake_backends)} with {len(fake_backends)} items")
print(f"   Access: backends[0].name = '{fake_backends[0].name}'")

print("\n2. backend() - Returns SINGLE")
print("   Usage: service.backend('ibm_kyoto')")
single = FakeKyoto()
print(f"   Returns: {type(single)}")
print(f"   Access: backend.name = '{single.name}'")

print("\n3. least_busy() - Returns SINGLE (auto-selected)")
print("   Usage: service.least_busy(simulator=False)")
print(f"   Returns: {type(single)}")
print("   Selection: Backend with shortest queue")

print("\n" + "=" * 60)
print("WHEN TO USE EACH")
print("=" * 60)
print("\n  backends()   → Need list to iterate/filter further")
print("  backend()    → Know exact backend name")
print("  least_busy() → Want automatic selection")

print("\n✅ Exam tip: Check return type - LIST vs SINGLE object")

In [ ]:
# ============================================================
# CHALLENGE 1: Backend Selection Logic
# ============================================================
# Task: Write a function that selects an appropriate backend
# Expected: Returns backend based on requirements
# ============================================================

def select_backend(backends_list, min_qubits=None, prefer_name=None):
    """
    Select appropriate backend from list.
    
    Args:
        backends_list: List of backends (simulates service.backends())
        min_qubits: Minimum qubit requirement
        prefer_name: Preferred backend name (partial match)
    
    Returns:
        Selected backend or None
    """
    # YOUR CODE HERE
    # Filter by min_qubits if specified
    candidates = backends_list
    
    if min_qubits:
        candidates = [b for b in candidates if b.num_qubits >= min_qubits]
    
    if prefer_name:
        preferred = [b for b in candidates if prefer_name.lower() in b.name.lower()]
        if preferred:
            return preferred[0]
    
    return candidates[0] if candidates else None

# Test your solution
test_backends = [FakeManilaV2(), FakeKyoto()]

# Test 1: Get any backend
result1 = select_backend(test_backends)
print(f"Test 1 - Any backend: {result1.name}")
assert result1 is not None, "Should return a backend"

# Test 2: With min_qubits
result2 = select_backend(test_backends, min_qubits=100)
print(f"Test 2 - Min 100 qubits: {result2.name if result2 else 'None'}")
assert result2.num_qubits >= 100, "Should meet qubit requirement"

# Test 3: With preferred name
result3 = select_backend(test_backends, prefer_name='kyoto')
print(f"Test 3 - Prefer 'kyoto': {result3.name if result3 else 'None'}")
assert 'kyoto' in result3.name.lower(), "Should match preferred name"

print("\n✅ Challenge 1 PASSED!")

In [ ]:
# ============================================================
# CHALLENGE 2: Run Circuit on Backend
# ============================================================
# Task: Create a Bell state circuit and run with Sampler
# Expected: Verify Bell state measurement distribution
# ============================================================

def run_bell_state(backend):
    """
    Create Bell state and run on backend with Sampler.
    
    Args:
        backend: Backend to run on
    
    Returns:
        Measurement counts dictionary
    """
    # Create Bell state circuit
    qc = QuantumCircuit(2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure_all()
    
    # Transpile for backend
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    transpiled = pm.run(qc)
    
    # Run with Sampler
    sampler = Sampler(backend)
    job = sampler.run([transpiled], shots=1000)
    result = job.result()
    
    # Get counts from BitArray
    counts = result[0].data.meas.get_counts()
    return counts

# Test your solution
backend = FakeKyoto()
counts = run_bell_state(backend)

print("Bell State Measurement Results:")
print("=" * 40)
for bitstring, count in sorted(counts.items()):
    print(f"  |{bitstring}⟩: {count}")

# Verify Bell state (should see mostly 00 and 11)
total = sum(counts.values())
bell_states = counts.get('00', 0) + counts.get('11', 0)
bell_fraction = bell_states / total

print(f"\nBell states (00 + 11): {bell_states}/{total} = {bell_fraction:.1%}")
assert bell_fraction > 0.9, "Should be mostly Bell states"

print("\n✅ Challenge 2 PASSED - Bell state created and measured!")